The project is Resume Coach with AI. I have created this capstone project that will help users with getting jobs that they prefer. The user can upload ATS friendly resume and also it will take the user's preference into consideration.
Here, the example I have taken is a resume of a DevOps Engineer. It will also receive a list of jobs uploaded on LinkedIn in the past 24 hrs. I have set the location to global.  
If the user is interested in chosing another role such as a cyber security analyst, it will provide a list of courses , certifications and projects that the user can learn to apply for the role.

Here we are installing the necessary libraries Pdfplumber is a function that is used to read the pdf uploaded which is used to read each page of the resume and combine the extracted text for further analysis.
We also install sentence-transformers for vector similarity.
We also install ipywidgets for creating interactive UI controls in our notebook.

In [ ]:
!pip install pdfplumber
# Uninstall packages from Kaggle base image that are not needed.
!pip uninstall -qy jupyterlab jupyterlab-lsp
# Install the google-genai SDK for this codelab.
!pip install -qU 'google-genai==1.7.0'
!pip install -qU sentence-transformers
!pip install -q ipywidgets

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import os
import numpy as np # linear algebra
import json
import ast
import re
import requests
import torch
import pdfplumber
import ipywidgets as widgets
import pandas as pd
from sentence_transformers import SentenceTransformer, util # Input data files are available in the read-only "../input/" directory
from datetime import datetime
import google.generativeai as genai
from IPython.display import display


****To run the following cell, your API key must be stored it in a Kaggle secret named GOOGLE_API_KEY.If you don't already have an API key, you can grab one from AI Studio. You can find detailed instructions in the docs.To make the key available through Kaggle secrets, choose Secrets from the Add-ons menu and follow the instructions to add your key or enable it for this notebook.****

In [ ]:
from kaggle_secrets import UserSecretsClient
import google.generativeai as genai

# 🔐 Securely fetch the API key from Kaggle Secrets
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

# ✅ Configure the Gemini client
genai.configure(api_key=GOOGLE_API_KEY)


model = genai.GenerativeModel("gemini-pro")

In [ ]:


# Walk through input directory to find your file
for root, dirs, files in os.walk("/kaggle/input"):
    for file in files:
        print(os.path.join(root, file))

In [ ]:
# Create an interactive widget for user input on job preferences/interests.
interests_widget = widgets.Text(
    value="",
    placeholder="Enter job preferences (e.g. remote, data science)",
    description="Interests:",
    disabled=False
)
display(interests_widget)

In [ ]:
# Wait for user input. (User can type in their preferences; then re-run the next cell.)
user_interests = interests_widget.value

if user_interests.strip():
    print("User Preferences:", user_interests)
else:
    print("No additional preferences provided.")

In [ ]:
import pdfplumber
import json
import re
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from sentence_transformers import SentenceTransformer
from google.generativeai import GenerativeModel
def extract_resume_text(pdf_path):
    text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text
    except Exception as e:
        print("Error reading PDF:", e)
    return text

resume_path = '/kaggle/input/resume-pdf/Sijumary Johnson - SRE Resume (2) (1).pdf'
resume_text = extract_resume_text(resume_path)

# === STEP 2: Create Embedding for Resume (optional for later comparison use) ===
semantic_model = SentenceTransformer('all-MiniLM-L6-v2')
resume_embedding = semantic_model.encode(resume_text, convert_to_tensor=True)

# === STEP 3: Predict Target Role Based on Resume ===
model = GenerativeModel("models/gemini-1.5-pro")  # Ensure you're using a proper wrapper if needed
role_prompt = f"""
You are an expert career analyst. Analyze the following resume text and suggest the most suitable job role for this candidate.
Return only the job title.

Resume:
{resume_text}
"""

role_response = model.generate_content(role_prompt)
target_position = role_response.text.strip()
print(f"🎯 Suggested Target Position: {target_position}")

# === STEP 4: Generate Recommendations for That Role ===
qualification_prompt = f"""
You are an expert career counselor. Based on the target job position provided, suggest 3 projects, 3 online courses, 
and 3 professional certifications that a candidate should pursue to be qualified for the role. 
Return the output as valid JSON with keys "projects", "courses", "certifications".

Target Job Position: {target_position}
"""

qualification_response = model.generate_content(qualification_prompt)
qualifications_json = qualification_response.text
print("📦 Raw Recommendations:\n", qualifications_json)

# === STEP 5: Clean + Parse JSON Output ===
cleaned_json = re.sub(r"^```(?:json)?|```$", "", qualifications_json.strip(), flags=re.MULTILINE).strip()
qualifications = json.loads(cleaned_json)

# === STEP 6: Create Card Widgets ===
def create_cards(items, item_type):
    cards = []
    for item in items:
        title = item.get("title", "Untitled")
        description = item.get("description", "No description available.")
        provider = item.get("platform") or item.get("certifying_body") or item.get("provider", "")
        skills = item.get("skills_demonstrated", [])
        
        # Create Google search fallback link
        link = f"https://www.google.com/search?q={title.replace(' ', '+')}"

        title_html = f'<a href="{link}" target="_blank"><b>{title}</b></a>'
        skills_html = "<ul>" + "".join(f"<li>{skill}</li>" for skill in skills) + "</ul>" if skills else ""

        card_html = f"""
        <div style="padding:10px; border:1px solid #ccc; border-radius:10px; margin:5px;">
            {title_html}<br>
            <p>{description}</p>
            <small><i>{provider}</i></small>
            {skills_html}
        </div>
        """
        cards.append(widgets.HTML(value=card_html))
    return widgets.VBox(cards)

# === STEP 7: Build and Display the Tabbed Dashboard ===
projects_tab = create_cards(qualifications.get("projects", []), "projects")
courses_tab = create_cards(qualifications.get("courses", []), "courses")
certifications_tab = create_cards(qualifications.get("certifications", []), "certifications")

tab = widgets.Tab(children=[projects_tab, courses_tab, certifications_tab])
tab.set_title(0, "📁 Projects")
tab.set_title(1, "🎓 Courses")
tab.set_title(2, "📜 Certifications")

display(tab)


**User Customization:– We use an ipywidget to create a text box that allows the user to enter job preferences.– The widget’s value (user_interests) is later added to our Gemini prompt so the suggestions are tailored further.**


* This prompt instructs the Gemini model to analyze the resume and suggest 3 career paths with reasons, salary estimates, and recommended skills.Send the prompt to Gemini and receive career suggestions.
* We use the generate_content() function to get AI-generated career advice in JSON format based on the resume.

Send the prompt to Gemini and receive career suggestions
We use the generate_content() function to get AI-generated career advice in JSON format based on the resume.
Prompt Engineering:
 We construct a detailed few-shot prompt including an example and the resume text.
 If the user provided additional interests, that context is appended, tailoring the career advice even further.

In [ ]:
# Append user interests to the prompt if provided.
extra_context = ""
if user_interests.strip() != "":
    extra_context = f"\nCandidate's specific interests: {user_interests}\n"
# Define few-shot prompt
few_shot_prompt = """
You are a helpful career advisor. Given a candidate's resume, suggest 3 suitable career paths based on their skills and experience.
Return the output as valid JSON with the following structure:
[
  {
    "job_title": "...",
    "match_reason": "...",
    "estimated_salary_range": "...",
    "recommended_skills_to_learn": [...]
  }
]

Example:
Resume: "I am a software developer with 3 years experience in Python and Django."

Output:
[
  {
    "job_title": "Backend Engineer",
    "match_reason": "Strong backend skills in Python and Django",
    "estimated_salary_range": "$80k-$100k",
    "recommended_skills_to_learn": ["Docker", "REST APIs"]
  },
  {
    "job_title": "Python Developer",
    "match_reason": "Expertise in Python, experience in Django",
    "estimated_salary_range": "$75k-$95k",
    "recommended_skills_to_learn": ["Flask", "CI/CD"]
  },
  {
    "job_title": "Full Stack Developer",
    "match_reason": "Good backend skills, can upskill on frontend",
    "estimated_salary_range": "$85k-$105k",
    "recommended_skills_to_learn": ["React", "GraphQL"]
  }
]

Now use this resume:
""" + resume_text

# Create a model
model = genai.GenerativeModel('models/gemini-1.5-pro')

# Generate career suggestions
response = model.generate_content(few_shot_prompt)
career_advice_json = response.text

print(career_advice_json)

Output Visualization:
The JSON output from Gemini is parsed and pretty-printed.
 We also convert it into a Pandas DataFrame for an easy-to-read table view.

In [ ]:
try:
    structured_output = json.loads(career_advice_json)
    pretty_json = json.dumps(structured_output, indent=2)
    print("Parsed and Pretty-Printed JSON Output:\n", pretty_json)
    df = pd.DataFrame(structured_output)
    print("\nCareer Suggestions Table:")
    display(df)
except Exception as e:
    print("Error parsing JSON:", e)
    print("Raw Gemini Response:\n", career_advice_json)

In [ ]:
from kaggle_secrets import UserSecretsClient
secrets = UserSecretsClient()
RAPIDAPI_KEY = secrets.get_secret("RAPIDAPI_KEY")

In [ ]:
# Combine resume text + user interests
combined_input = f"""
Resume: {resume_text}
User Interests: {user_interests}

From the resume and user interests, generate 5 highly relevant job search keywords.
Return as a Python list of strings.
"""

# Generate keywords using Gemini
keyword_response = model.generate_content(combined_input)
keywords = keyword_response.text.strip()

# Print/parse
print("Suggested Keywords:", keywords)

In [ ]:
import ast

# First, print the raw Gemini output to make sure it's valid
print("🔍 Raw Gemini output:", keywords)

# Remove any markdown formatting (backticks, code fences)
cleaned_keywords = keywords.strip().strip("`").replace("python", "").replace("```", "").strip()

# Then safely parse the cleaned string
try:
    keyword_list = ast.literal_eval(cleaned_keywords)
    print("✅ Parsed keyword list:", keyword_list)
except Exception as e:
    print("❌ Failed to parse keyword list:", e)
    keyword_list = ["DevOps Engineer"]  # Fallback in case of error

In [ ]:
url = "https://linkedin-data-api.p.rapidapi.com/search-jobs-v2"
headers = {
    "X-RapidAPI-Key": RAPIDAPI_KEY,
    "X-RapidAPI-Host": "linkedin-data-api.p.rapidapi.com"
}
querystring = {
    "keywords": keyword_list[0],  # Now guaranteed to work
    "locationId": "92000000",  # Remote
    "datePosted": "past24Hours",
    "sort": "mostRelevant"
}

response = requests.get(url, headers=headers, params=querystring)
job_items = response.json().get("data", []) if response.status_code == 200 else []

In [ ]:
semantic_model = SentenceTransformer('all-MiniLM-L6-v2')
resume_embedding = semantic_model.encode(resume_text, convert_to_tensor=True)

def compute_semantic_match(description, resume_embedding):
    if not description or not isinstance(description, str):
        return 0.0
    job_embedding = semantic_model.encode(description, convert_to_tensor=True)
    similarity = util.pytorch_cos_sim(resume_embedding, job_embedding).item()
    return round(similarity * 100, 2)

def convert_timestamp(ts):
    if ts:
        return datetime.utcfromtimestamp(ts / 1000).strftime('%Y-%m-%d')
    return "N/A"

In [ ]:
job_list = []
for job in job_items:
    description = job.get("description", "")
    match_score = compute_semantic_match(description, resume_embedding)
    job_list.append({
        "Job Title": job.get("title", "N/A"),
        "Company": job.get("company", {}).get("name", "N/A"),
        "Location": job.get("location", "N/A"),
        "Posted Date": convert_timestamp(job.get("postedTimestamp", 0)),
        "Job URL": job.get("url", "N/A"),
        "Match Confidence (%)": match_score
    })

In [ ]:
df_jobs = pd.DataFrame(job_list)
df_jobs = df_jobs.sort_values("Match Confidence (%)", ascending=False)
display(df_jobs.head(20))

In [ ]:
target_position = user_interests.strip() if user_interests.strip() != "" else keyword_list[0]

qualification_prompt = f"""
You are an expert career counselor. Based on the target job position provided, suggest 3 projects, 3 online courses, 
and 3 professional certifications that a candidate should pursue to be qualified for the role. 
Return the output as valid JSON with keys "projects", "courses", "certifications".

Target Job Position: {target_position}
"""

qualification_response = model.generate_content(qualification_prompt)
qualifications_json = qualification_response.text
print("Raw Recommendations Response:\n", qualifications_json)

try:
    qualifications = json.loads(qualifications_json)
    # Convert the recommendations to a DataFrame for a dashboard view
    recommendations_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in qualifications.items()]))
    display(recommendations_df)
except Exception as e:
    print("Error parsing qualification recommendations:", e)

In [ ]:
cleaned_json = re.sub(r"^```(?:json)?|```$", "", qualifications_json.strip(), flags=re.MULTILINE).strip()

try:
    qualifications = json.loads(cleaned_json)
    print("✅ Qualifications successfully parsed!")
except Exception as e:
    print("❌ Error parsing cleaned response:", e)
    print("Cleaned Response:\n", cleaned_json)